<a href="https://colab.research.google.com/github/NazarioR9/BNBR_Challenge/blob/master/notebooks/MLMBertBaseGenericModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Specs

In [ ]:
!nvidia-smi

Thu Jul  2 18:02:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Install

In [ ]:
!pip install transformers catalyst --quiet

In [ ]:
!pip install torchcontrib --quiet

#Imports

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
import os, sys, gc
import time
import string
import subprocess
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, f1_score, accuracy_score
from sklearn.metrics import classification_report

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
import transformers
from transformers import AutoConfig, AutoTokenizer, BartConfig, RobertaTokenizer

In [ ]:
sys.path.append('../bnbr')

In [ ]:
import bnbr
from bnbr.activation import *
from bnbr.data import *
from bnbr.models import *
from bnbr.loss import *
from bnbr.utils import *

Using TensorFlow backend.


#Envs

In [ ]:
!unzip ../mlm_finetuned_models/mlm_bert_base_

In [ ]:
seed = 2020
seeds = [42, 5, 12]
n_folds = 10
model_name = 'bert-base-uncased'
model_path = 'mlm_bert_base_/'
dirs = ['models/', f'evals/{model_name}']
exts = ['.csv', '.pkl']

In [ ]:
wm = WorkplaceManager(seed, dirs, exts)

Set seed to 2020.
Deleted models/ evals/bert-base-uncased
Deleted .csv .pkl
Created models/ evals/bert-base-uncased


#Tokenizer

In [ ]:
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Data

In [ ]:
path = '../data/'

In [ ]:
os.listdir(path)

['final_test.csv',
 'final_train.csv',
 'Train.csv',
 'README.md',
 'test_clean.csv',
 'train_clean.csv',
 'Test.csv',
 'final_train_v2.csv',
 'final_test_v3.csv',
 'train_translated_to_french.csv',
 'final_train_v3.csv',
 'final_test_v2.csv',
 'test_translated_to_french.csv',
 'SampleSubmission.csv']

In [ ]:
train = pd.read_csv(path+'final_train.csv')
test = pd.read_csv(path+'final_test.csv')
sample = pd.read_csv(path+'SampleSubmission.csv')

In [ ]:
train.shape

(597, 7)

In [ ]:
train.head()

,ID,text,label,Depression,Alcohol,Suicide,Drugs
0,SUAVK39Z,i feel that it was better i die am happy,0,1,0,0,0
1,9JDAGUV3,why do i get hallucinations ?,3,0,0,0,1
2,419WR1LQ,i am stressed due to lack of financial suppor...,0,1,0,0,0
3,6UY7DX6Q,why is life important ?,2,0,0,1,0
4,FYC0FTFB,how could i be helped to go through the depre...,0,1,0,0,0


In [ ]:
train = train[['text', 'Depression', 'Alcohol', 'Suicide', 'Drugs', 'label']]

In [ ]:
train['length'] = train['text'].apply(lambda x: len(tokenizer.encode(x)))
test['length'] = test['text'].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
train.head()

,text,Depression,Alcohol,Suicide,Drugs,label,length
0,i feel that it was better i die am happy,1,0,0,0,0,12
1,why do i get hallucinations ?,0,0,0,1,3,10
2,i am stressed due to lack of financial suppor...,1,0,0,0,0,13
3,why is life important ?,0,0,1,0,2,7
4,how could i be helped to go through the depre...,1,0,0,0,0,13


In [ ]:
train.label.value_counts()

0    344
1    132
2     65
3     56
Name: label, dtype: int64

#Split in folds

In [ ]:
train.head()

,text,Depression,Alcohol,Suicide,Drugs,label,length
0,i feel that it was better i die am happy,1,0,0,0,0,12
1,why do i get hallucinations ?,0,0,0,1,3,10
2,i am stressed due to lack of financial suppor...,1,0,0,0,0,13
3,why is life important ?,0,0,1,0,2,7
4,how could i be helped to go through the depre...,1,0,0,0,0,13


In [ ]:
kf = StratifiedKFold(n_folds)
#train['fold'] = 0
for fold, (tr_index, val_index) in enumerate(kf.split(train.index, train.label)):
  train.loc[val_index, 'fold'] = fold

In [ ]:
train.head()

,text,Depression,Alcohol,Suicide,Drugs,label,length,fold
0,i feel that it was better i die am happy,1,0,0,0,0,12,0.0
1,why do i get hallucinations ?,0,0,0,1,3,10,0.0
2,i am stressed due to lack of financial suppor...,1,0,0,0,0,13,0.0
3,why is life important ?,0,0,1,0,2,7,0.0
4,how could i be helped to go through the depre...,1,0,0,0,0,13,0.0


#Utilities

In [ ]:
class GlobalConfig:
  accumulate_grad_batches = 1
  activation = torch.softmax
  batch_size = 32
  clip_grad = False
  epochs = 10
  folds = n_folds
  hidden_multiplier = 1
  high_dropout = 0.5
  id = 0
  L = 1
  loss = MHMLoss('ce', 10)
  loss_name = 'ce'
  low_dropout = 0.3
  lr = 4e-5
  max_grad_norm = 1.0
  max_tokens = 50
  mhm_activation = Mish()
  model_name = model_name
  model_path = model_path
  multi_samp_nb = 3
  n = 11
  output_hidden_states = True
  reinit = False
  scheduler = False
  swa = False
  swa_start = 4
  swa_freq = 1
  swa_lr = 3e-5
  task = 'train'
  warmup_steps = 0
  

  def __init__(self, xp_nb=0):
    self.xp_nb = xp_nb
    self._name()
    self._lightweight()

  def update(self, **kwargs):
    for name in kwargs.keys():
      if hasattr(self, name):
        setattr(self, name, kwargs[name])

  def _lightweight(self):
    self.fold = 0
    self.train_df = None
    self.test_df = None
    self.val_df = None

  def save(self, path=''):
    self._lightweight()
    
    with open(path+self.name, 'wb') as f:
      pickle.dump(self, f)
      f.close()

  def _name(self):
    if not hasattr(self, 'name'):
      vocab = string.ascii_letters + string.digits
      for i in range(self.xp_nb):
        n = ''
        for _ in range(10):
          n += vocab[random.randint(0, len(vocab)-1)]

      self.name = 'config_'+n+'.pkl'

In [ ]:
global global_config
global_config = GlobalConfig()

In [ ]:
def run_fold(score_pkl='cv_score.pkl'):
  fold_scores = []
  print_dict_list = {}

  try:
    with open(score_pkl, 'rb') as f:
      cv_score = pickle.load(f)
      f.close()
  except FileNotFoundError:
    cv_score = 0

  completed = len(os.listdir('models/'))

  for fold in train.fold.unique():
    if fold < completed: continue

    df_train = train[train.fold != fold].copy()
    df_val = train[train.fold == fold].copy()
    print_dict_list[fold] = []
    best_score = np.inf

    global_config.update(
      fold = fold,
      train_df=df_train,
      val_df=df_val
    )


    trainer = Trainer(global_config)
    es = EarlyStopping()
    
    for epoch in range(global_config.epochs):
      trainer.fit_one_epoch(epoch)
      # es.update(trainer.scores[epoch][-1])

      # if es.stop and epoch >=5:
      #   break

    trainer.save_best_eval()
    trainer.evaluate_post_processing()

    cv_score += trainer.best_log

    os.system(f"rm {score_pkl}")
    with open(score_pkl, 'wb') as f:
      pickle.dump(cv_score, f)
      f.close()

    trainer = None, None
    del trainer, df_train, df_val
    torch.cuda.empty_cache()
    gc.collect()


In [ ]:
def run_prediction(path='models/'):
  preds = []
  global_config.update(
      task='test',
      test_df=test
    )

  for fold in train.fold.unique():

    module = LightTrainingModule(global_config)
    module.load_state_dict(torch.load(path+f'model_{fold}.bin'))
    trainer = Trainer(global_config, module=module)
    
    trainer.predict()
    pred = trainer.get_preds()

    preds.append(pred)

    module, trainer = None, None
    del trainer, module
    torch.cuda.empty_cache()
    gc.collect()

  return preds

#The Torch Way

##Run folds

In [ ]:
gc.collect()

0

In [ ]:
run_fold()

____________________________________________________________________________________________________ 
Fold  9.0
	____________________________________________________________________________________________________
	| Epoch: 0 - Loss: 1.09150 - ValLoss: 0.97977 - Log: 0.97977 - F1: 0.61239 - LogF1: 0.36029
	____________________________________________________________________________________________________
	| Epoch: 1 - Loss: 0.69432 - ValLoss: 0.60898 - Log: 0.60898 - F1: 0.70884 - LogF1: 0.54993
	____________________________________________________________________________________________________
	| Epoch: 2 - Loss: 0.39188 - ValLoss: 0.47141 - Log: 0.47141 - F1: 0.81701 - LogF1: 0.67280
	____________________________________________________________________________________________________
	| Epoch: 3 - Loss: 0.20896 - ValLoss: 0.43959 - Log: 0.43959 - F1: 0.88294 - LogF1: 0.72167
	____________________________________________________________________________________________________
	| Epo

In [ ]:
# wm.clear(['evals/', 'models/'])

# CrossVal Score

In [ ]:
cvl = CrossValLogger(train, n_folds, path=f'evals/{model_name}/')

In [ ]:
score1, score2 = cvl.show_results(True)

OOF_CV_SCORE: 0.36311 | OVR_SCORE: 0.35718


In [ ]:
gc.collect()

30

#Save config

In [ ]:
global_config.save()

##Prediction

In [ ]:
%%time
preds = run_prediction()

CPU times: user 27.3 s, sys: 7.31 s, total: 34.6 s
Wall time: 2min 39s


In [ ]:
raw_outputs = np.mean(preds, axis=0)

#Submission

In [ ]:
mkdir ../submissions

In [ ]:
spath = 'submissions/'

In [ ]:
sample[["Depression","Alcohol","Suicide","Drugs"]] = raw_outputs
sample.to_csv(spath + f'1_{model_name.split("/")[-1]}__{global_config.name.split(".")[0]}_{score1:.5f}_{score2:.5f}.csv', index=False)

In [ ]:
sample.head()

,ID,Depression,Alcohol,Suicide,Drugs
0,02V56KMO,0.505766,0.034466,0.443683,0.016085
1,03BMGTOK,0.990849,0.001714,0.005640,0.001797
2,03LZVFM6,0.991154,0.001594,0.005319,0.001934
3,0EPULUM5,0.990684,0.001690,0.005642,0.001984
4,0GM4C5GD,0.026632,0.154714,0.034871,0.783782


In [ ]:
sample.describe()

,Depression,Alcohol,Suicide,Drugs
count,309.000000,309.000000,309.000000,309.000000
mean,0.554633,0.215247,0.137561,0.092558
std,0.445833,0.373768,0.259679,0.240964
min,0.003956,0.001534,0.004576,0.001772
25%,0.012706,0.001991,0.006455,0.002265
50%,0.789282,0.023294,0.010033,0.013644
75%,0.988923,0.108046,0.079445,0.030495
max,0.991854,0.974116,0.915662,0.934257


In [ ]:
sample[["Depression","Alcohol","Suicide","Drugs"]].apply(np.argmax, axis=1).value_counts()

0    176
1     64
2     44
3     25
dtype: int64